![Image](./img/dataset-diagram-logo.png)

**objectif** : puissance de pandas en N dimentions, avec labels !! </br>
-> outils **pandas-compatibles pour tableaux multidimentionnels**

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
truc = xr.DataArray(np.random.randint(2, 3))
truc

<xarray.DataArray ()>
array(2)

In [3]:
data = xr.DataArray(np.random.randint(6, size=(2, 3)), coords={'x': ['a', 'b']}, dims=('x', 'y'))
data

<xarray.DataArray (x: 2, y: 3)>
array([[5, 3, 3],
       [0, 2, 4]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

In [4]:
# On peut lui donner du pandas
xr.DataArray(pd.Series(range(3), index=list('abc'), name='foo'))

<xarray.DataArray 'foo' (dim_0: 3)>
array([0, 1, 2])
Coordinates:
  * dim_0    (dim_0) object 'a' 'b' 'c'

## Propriétés de DataArray

In [5]:
print('values : ', data.values)
print('dims : ', data.dims)
print('coords : ', data.coords)
data.attrs  # pour métadonnées

values :  [[5 3 3]
 [0 2 4]]
dims :  ('x', 'y')
coords :  Coordinates:
  * x        (x) <U1 'a' 'b'


{}

## Indexing

In [6]:
#data[[0]]   #comme dans numpy, avec index int
#data.loc['a':'b'] #comme dans pandas, avec des labels
# data.isel(x=slice(1))  #par slice : dimension + int
data.sel(x=['a', 'b']) # dimension + label

# print(data)
# print(data.isel(x=slice(3)))

<xarray.DataArray (x: 2, y: 3)>
array([[5, 3, 3],
       [0, 2, 4]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

## Computation

In [7]:
# très similaire numpy
# data + 10
# np.sin(data)
# data.T
# data.sum()

# mais on peut utiliser les noms des axes au lieu de leur numéro
data.mean(dim='y')

<xarray.DataArray (x: 2)>
array([3.66666667, 2.        ])
Coordinates:
  * x        (x) <U1 'a' 'b'

In [8]:
# Plus besoin de gerer l'ajout de nouvel ax (new_axis)
a = xr.DataArray(np.random.randint(3, size=(3)), [data.coords['y']])
b = xr.DataArray(np.random.randint(4, size=(4)), dims='z')

print('a : ', a)
print('b : ', b)
print('a+b : ', a+b)

a :  <xarray.DataArray (y: 3)>
array([2, 0, 2])
Coordinates:
  * y        (y) int64 0 1 2
b :  <xarray.DataArray (z: 4)>
array([0, 0, 2, 3])
Dimensions without coordinates: z
a+b :  <xarray.DataArray (y: 3, z: 4)>
array([[2, 2, 4, 5],
       [0, 0, 2, 3],
       [2, 2, 4, 5]])
Coordinates:
  * y        (y) int64 0 1 2
Dimensions without coordinates: z


In [9]:
data - data.T

<xarray.DataArray (x: 2, y: 3)>
array([[0, 0, 0],
       [0, 0, 0]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

In [10]:
data[:-1] - data[:1]

<xarray.DataArray (x: 1, y: 3)>
array([[0, 0, 0]])
Coordinates:
  * x        (x) <U1 'a'
Dimensions without coordinates: y

## GroupBy

In [11]:
display(data)
labels = xr.DataArray(['E', 'F', 'E'], [data.coords['y']], name='labels')

# data1 = data.groupby(labels).mean('y')
# data1
data2 = data.groupby(labels).apply(lambda x: x - x.min())
data2
# print('data : \n', data)
# print('\n')
# print('labels : \n', labels)
# print('\n')
# print('data1 : \n', data1)
# print('\n')
# print('data2 : \n', data2)


<xarray.DataArray (x: 2, y: 3)>
array([[5, 3, 3],
       [0, 2, 4]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

<xarray.DataArray (x: 2, y: 3)>
array([[5, 1, 3],
       [0, 0, 4]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

## Pandas

In [12]:
series = data.to_series()
series.to_xarray()

<xarray.DataArray (x: 2, y: 3)>
array([[5, 3, 3],
       [0, 2, 4]])
Coordinates:
  * x        (x) object 'a' 'b'
  * y        (y) int64 0 1 2

## Dataset

In [13]:
ds = xr.Dataset({'foo': data, 'bar': ('x', [1, 2]), 'baz': np.pi})
ds

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y
Data variables:
    foo      (x, y) int64 5 3 3 0 2 4
    bar      (x) int64 1 2
    baz      float64 3.142

In [14]:
ds['foo']

<xarray.DataArray 'foo' (x: 2, y: 3)>
array([[5, 3, 3],
       [0, 2, 4]])
Coordinates:
  * x        (x) <U1 'a' 'b'
Dimensions without coordinates: y

## NetCDF

In [15]:
ds.to_netcdf('example.nc')

In [16]:
xr.open_dataset('example.nc')

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) object 'a' 'b'
Dimensions without coordinates: y
Data variables:
    foo      (x, y) int64 ...
    bar      (x) int64 ...
    baz      float64 ...